In [21]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D,Convolution1D,MaxPooling1D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf


In [23]:
true_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//true_array.npy')
false_stand_data=np.load(r'F:\\深度学习模型项目\\神经网络GRU模型\\data3\\data3//false_array.npy')

true_stand_data=true_stand_data[:,:,[1,2,6],0]
false_stand_data=false_stand_data[:,:,[1,2,6],0]

# true_stand_data=true_stand_data.swapaxes(1, 2)
# false_stand_data=false_stand_data.swapaxes(1, 2)

In [24]:
true_stand_data[0].shape

(110, 3)

In [25]:
sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }

true_sample_channel_1 = []
false_sample_channel_1 = []


for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))

In [26]:
combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)

combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()

In [35]:
from sklearn.model_selection import train_test_split
train_X,test_X, train_y, test_y = train_test_split(combin_train_data1,
                                                   combin_target_data,
                                                   test_size = 0.2,
                                                   random_state = 0)

In [30]:
# Model 1
in_1 = Input(shape=(110,3))
C_1 = Convolution1D(filters=12,kernel_size=3,input_shape=(110,3),padding='same')(in_1)
C_1 = Convolution1D(filters=12,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=36,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=36,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=108,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=108,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=240,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=240,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)


x_middle_concat = GRU(units=480,return_sequences=True)(C_1)
x_middle_concat = GRU(units=100)(x_middle_concat)

model_final_dense_out = Dense(2, activation='softmax')(x_middle_concat)

model = Model(inputs=in_1, outputs=model_final_dense_out)
print(model.summary())

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 110, 3)]          0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 110, 12)           120       
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 110, 12)           444       
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 55, 12)            0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 55, 36)            1332      
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 55, 36)            3924      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 27, 36)          

In [28]:
import os
if 'GRU_predict_v10_2.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v10_2.h5")

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=30
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v10_2.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.8,
        patience=5, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy']
             
             )

model.fit(x=train_X,
          y=train_y,
          batch_size=36,
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list,
          validation_data=(test_X,test_y)
         )



Epoch 1/100
5408/5408 [==============================] - 275s 51ms/step - loss: 0.6671 - accuracy: 0.6080
Epoch 2/100
5408/5408 [==============================] - 285s 53ms/step - loss: 0.6463 - accuracy: 0.6322
Epoch 3/100
5408/5408 [==============================] - 292s 54ms/step - loss: 0.6161 - accuracy: 0.6666
Epoch 4/100
5408/5408 [==============================] - 279s 52ms/step - loss: 0.5997 - accuracy: 0.6830
Epoch 5/100
5408/5408 [==============================] - 299s 55ms/step - loss: 0.5894 - accuracy: 0.6955
Epoch 6/100
5408/5408 [==============================] - 286s 53ms/step - loss: 0.5827 - accuracy: 0.6975
Epoch 7/100
5408/5408 [==============================] - 284s 52ms/step - loss: 0.5785 - accuracy: 0.7017
Epoch 8/100
5408/5408 [==============================] - 290s 54ms/step - loss: 0.5735 - accuracy: 0.7077
Epoch 9/100
5408/5408 [==============================] - 279s 52ms/step - loss: 0.5712 - accuracy: 0.7102
Epoch 10/100
5408/5408 [======================

KeyboardInterrupt: 

In [ ]:
import keras
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy

from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D,Convolution1D,MaxPooling1D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np
import tensorflow as tf


true_stand_data=np.load(r'.//data3//true_array.npy')
false_stand_data=np.load(r'.//data3//false_array.npy')

true_stand_data=true_stand_data[:,:,[1,2,6],0]
false_stand_data=false_stand_data[:,:,[1,2,6],0]


sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }

true_sample_channel_1 = []
false_sample_channel_1 = []


for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:]))
    
for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:]))
    
    
combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)

combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()    
    
# Model 1
in_1 = Input(shape=(110,3))
C_1 = Convolution1D(filters=12,kernel_size=3,input_shape=(110,3),padding='same')(in_1)
C_1 = Convolution1D(filters=12,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=36,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=36,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=108,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=108,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)

C_1 = Convolution1D(filters=240,kernel_size=3,padding='same')(C_1)
C_1 = Convolution1D(filters=240,kernel_size=3,padding='same')(C_1)
C_1 = MaxPooling1D(pool_size=2,strides=2, padding='valid')(C_1)


x_middle_concat = GRU(units=480,return_sequences=True)(C_1)
x_middle_concat = GRU(units=100)(x_middle_concat)

model_final_dense_out = Dense(2, activation='softmax')(x_middle_concat)

model = Model(inputs=in_1, outputs=model_final_dense_out)
print(model.summary())    
    
import os
if 'GRU_predict_v10_3.h5' in os.listdir('./'):
    model = load_model("./GRU_predict_v10_3.h5")

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=60
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict_v10_3.h5', 
    monitor='loss', 
    save_best_only=True),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.9,
        patience=50, 
        mode='auto',
        min_lr=0.00003)
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['val_accuracy']             
             )


from sklearn.model_selection import train_test_split
train_X,test_X, train_y, test_y = train_test_split(combin_train_data1,
                                                   combin_target_data,
                                                   test_size = 0.2,
                                                   random_state = 0)

model.fit(x=combin_train_data1,
          y=combin_target_data,
          batch_size=360,
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list,
#           validation_data=(test_X,test_y)
          validation_split=0.2,
         )


    
    
    
    
    
